![EvalML Logo](images/evalml_logo.png)


# What is EvalML?
EvalML is an AutoML library that builds, optimizes, and evaluates machine learning pipelines using domain-specific objective functions. 

Combined with [Featuretools](https://featuretools.featurelabs.com) and [Compose](https://compose.featurelabs.com), EvalML can be used to create end-to-end machine learning solutions for classification and regression problems. 

# Quick Start

In [ ]:
import evalml
from evalml import AutoMLSearch

## Load Data

First, we load in the features and outcomes we want to use to train our model.

In [ ]:
X, y = evalml.demos.load_breast_cancer()

## Configure search

EvalML has many options to configure the pipeline search. At the minimum, we need to define an objective function. For simplicity, we will use the F1 score in this example. However, the real power of EvalML is in using domain-specific [objective functions](objectives/overview.ipynb) or [building your own](objectives/custom_objectives.ipynb).

Below EvalML utilizes Bayesian optimization (EvalML's default optimizer) to search and find the best pipeline defined by the given objective.

In [ ]:
automl = AutoMLSearch(problem_type="binary",
                      objective="f1",
                      max_pipelines=5)

In order to validate the results of the pipeline creation and optimization process, we will save some of our data as a holdout set.

In [ ]:
X_train, X_holdout, y_train, y_holdout = evalml.preprocessing.split_data(X, y, test_size=.2)

When we call `.search()`, the search for the best pipeline will begin. There is no need to wrangle with missing data or categorical variables as EvalML includes various preprocessing steps (like imputation, one-hot encoding, feature selection) to ensure you're getting the best results. As long as your data is in a single table, EvalML can handle it. If not, you can reduce your data to a single table by utilizing [Featuretools](https://featuretools.featurelabs.com) and its Entity Sets.

You can find more information on pipeline components and how to integrate your own custom pipelines into EvalML [here](pipelines/overview.ipynb).

In [ ]:
automl.search(X_train, y_train)

## See Pipeline Rankings

After the search is finished we can view all of the pipelines searched, ranked by score. Internally, EvalML performs cross validation to score the pipelines. If it notices a high variance across cross validation folds, it will warn you. EvalML also provides additional [data checks](data_checks/data_checks_in_automl.ipynb) to analyze your data to assist you in producing the best performing pipeline. 

In [ ]:
automl.rankings

## Describe pipeline

If we are interested in see more details about the pipeline, we can describe it using the `id` from the rankings table:

In [ ]:
automl.describe_pipeline(3)

## Select Best pipeline
We can now select best pipeline and score it on our holdout data:

In [ ]:
pipeline = automl.best_pipeline
pipeline.fit(X_train, y_train)
pipeline.score(X_holdout, y_holdout, ["f1"])

We can also visualize the structure of our pipeline:

In [ ]:
pipeline.graph()

## Whats next?

Head into the more in-depth automated walkthrough [here](automl/pipeline_search.ipynb) or any advanced topics below.

# Getting Started
[What is EvalML](self)

[Installation](install)

# Objective functions

[Overview](objectives/overview)

[Fraud Prediction](demos/fraud)

[Lead Scoring](demos/lead_scoring)

[Defining Custom Objectives](objectives/custom_objectives)

# Automated Machine Learning

[Setting up pipeline search](automl/pipeline_search)

[Exploring search results](automl/search_results)

[Regression Example](automl/regression_example)

[Overfitting Protection](automl/overfitting_protection)

# Pipelines and Components

[Pipelines](pipelines/overview)

[Components](pipelines/components)

[Custom Pipelines](pipelines/custom_pipelines)

# Data Checks

[Overview](data_checks/data_checks_overview)

[Using Data Checks in AutoML](data_checks/data_checks_in_automl)

# Resources
[Changelog](changelog)

[API Reference](api_reference)

[FAQ](faq)